In [ ]:
// Kotlin notebook
import jline.*
import jline.lang.*
import jline.lang.nodes.*
import jline.lang.processes.*
import jline.lang.constant.*
import jline.solvers.ctmc.*
import jline.solvers.mva.*
import jline.solvers.mam.*
import jline.solvers.jmt.*
import jline.util.matrix.*

In [ ]:
// Create the large multiclass network
val model = Network("MyNetwork")

// Block 1: Network nodes
val source = Source(model, "Source")
val queue1 = Queue(model, "Queue1", SchedStrategy.FCFS)  // WebServer
val queue2 = Queue(model, "Queue2", SchedStrategy.FCFS)  // Storage1
val queue3 = Queue(model, "Queue3", SchedStrategy.PS)    // Storage2
val queue4 = Queue(model, "Queue4", SchedStrategy.FCFS)  // Storage3
val sink = Sink(model, "Sink")

// Block 2: Job classes with different characteristics
val jobclass1 = OpenClass(model, "Class1", 0)  // High-priority requests
val jobclass2 = OpenClass(model, "Class2", 0)  // Medium-priority requests
val jobclass3 = OpenClass(model, "Class3", 0)  // Low-priority requests

In [ ]:
// Arrival processes - different arrival rates per class
source.setArrival(jobclass1, Exp.fitMean(5.0))  // Class1: mean inter-arrival = 5.0 (rate = 0.2)
source.setArrival(jobclass2, Exp.fitMean(8.0))  // Class2: mean inter-arrival = 8.0 (rate = 0.125)
source.setArrival(jobclass3, Exp.fitMean(7.0))  // Class3: mean inter-arrival = 7.0 (rate = 0.143)

In [ ]:
// Service processes at WebServer (Queue1) - front-end processing
queue1.setService(jobclass1, Exp.fitMean(0.3))  // Class1: fast processing
queue1.setService(jobclass2, Exp.fitMean(0.5))  // Class2: medium processing
queue1.setService(jobclass3, Exp.fitMean(0.6))  // Class3: slower processing

// Service processes at Storage1 (Queue2) - database access
queue2.setService(jobclass1, Exp.fitMean(1.1))  // Class1: complex queries
queue2.setService(jobclass2, Exp.fitMean(1.3))  // Class2: medium queries
queue2.setService(jobclass3, Exp.fitMean(1.5))  // Class3: simple queries

// Service processes at Storage2 (Queue3) - file system access with PS
queue3.setService(jobclass1, Exp.fitMean(2.0))  // Class1: large files
queue3.setService(jobclass2, Exp.fitMean(2.1))  // Class2: medium files
queue3.setService(jobclass3, Exp.fitMean(1.9))  // Class3: small files

// Service processes at Storage3 (Queue4) - cache access
queue4.setService(jobclass1, Exp.fitMean(1.5))  // Class1: cache miss handling
queue4.setService(jobclass2, Exp.fitMean(0.9))  // Class2: cache hits
queue4.setService(jobclass3, Exp.fitMean(2.3))  // Class3: cache updates

In [ ]:
// Block 3: Complex routing topology with feedback loops
val P = model.initRoutingMatrix()

// Class1 routing
P.set(jobclass1, jobclass1, source, queue1, 1.0)     // Source → WebServer
P.set(jobclass1, jobclass1, queue1, queue2, 0.25)    // WebServer → Storage1 (25%)
P.set(jobclass1, jobclass1, queue1, queue3, 0.25)    // WebServer → Storage2 (25%)
P.set(jobclass1, jobclass1, queue1, queue4, 0.25)    // WebServer → Storage3 (25%)
P.set(jobclass1, jobclass1, queue1, sink, 0.25)      // WebServer → Sink (25%)
P.set(jobclass1, jobclass1, queue2, queue1, 1.0)     // Storage1 → WebServer (feedback)
P.set(jobclass1, jobclass1, queue3, queue1, 1.0)     // Storage2 → WebServer (feedback)
P.set(jobclass1, jobclass1, queue4, queue1, 1.0)     // Storage3 → WebServer (feedback)

// Class2 routing
P.set(jobclass2, jobclass2, source, queue1, 1.0)     // Source → WebServer
P.set(jobclass2, jobclass2, queue1, queue2, 0.25)    // WebServer → Storage1 (25%)
P.set(jobclass2, jobclass2, queue1, queue3, 0.25)    // WebServer → Storage2 (25%)
P.set(jobclass2, jobclass2, queue1, queue4, 0.25)    // WebServer → Storage3 (25%)
P.set(jobclass2, jobclass2, queue1, sink, 0.25)      // WebServer → Sink (25%)
P.set(jobclass2, jobclass2, queue2, queue1, 1.0)     // Storage1 → WebServer (feedback)
P.set(jobclass2, jobclass2, queue3, queue1, 1.0)     // Storage2 → WebServer (feedback)
P.set(jobclass2, jobclass2, queue4, queue1, 1.0)     // Storage3 → WebServer (feedback)

// Class3 routing
P.set(jobclass3, jobclass3, source, queue1, 1.0)     // Source → WebServer
P.set(jobclass3, jobclass3, queue1, queue2, 0.25)    // WebServer → Storage1 (25%)
P.set(jobclass3, jobclass3, queue1, queue3, 0.25)    // WebServer → Storage2 (25%)
P.set(jobclass3, jobclass3, queue1, queue4, 0.25)    // WebServer → Storage3 (25%)
P.set(jobclass3, jobclass3, queue1, sink, 0.25)      // WebServer → Sink (25%)
P.set(jobclass3, jobclass3, queue2, queue1, 1.0)     // Storage1 → WebServer (feedback)
P.set(jobclass3, jobclass3, queue3, queue1, 1.0)     // Storage2 → WebServer (feedback)
P.set(jobclass3, jobclass3, queue4, queue1, 1.0)     // Storage3 → WebServer (feedback)

model.link(P)

In [ ]:
// Create multiple solvers for comparison
val solvers = arrayOf(
    CTMC(model, "seed", 23000, "cutoff", 1),  // May be infinite on this model
    MVA(model, "seed", 23000),
    MAM(model, "seed", 23000),
    JMT(model, "seed", 23000, "samples", 1000000)  // High sample count for accuracy
)

// Note: Some solvers commented out due to potential issues with this model
// FLD, SSA, NC may have convergence issues with complex feedback

In [ ]:
// Execute all solvers and collect results
for ((i, solver) in solvers.withIndex()) {
    println("\n=== SOLVER ${i+1}: ${solver.name} ===")
    try {
        val avgTable = solver.avgTable
        avgTable.print()
    } catch (e: Exception) {
        println("Solver failed: ${e.message}")
    }
}

This complex multi-class open network demonstrates:

1. **Web Service Architecture**:
   - **Source**: External request arrivals
   - **Queue1 (WebServer)**: Front-end processing with FCFS
   - **Queue2 (Storage1)**: Database access with FCFS
   - **Queue3 (Storage2)**: File system with PS scheduling
   - **Queue4 (Storage3)**: Cache system with FCFS
   - **Sink**: Request completion

2. **Multi-Class Workload**:
   - **Class1**: High-priority (λ=0.2, fast web processing)
   - **Class2**: Medium-priority (λ=0.125, balanced processing)
   - **Class3**: Low-priority (λ=0.143, slower web processing)
   - Different service requirements at each tier

3. **Complex Routing Pattern**:
   - All classes enter through WebServer
   - 25% probability to each storage system
   - 25% probability to exit (complete)
   - 100% feedback from storage back to WebServer
   - Creates loops and multiple visits

4. **Service Differentiation**:
   - **WebServer**: Class1 fastest, Class3 slowest
   - **Storage1**: Class1 most complex queries
   - **Storage2**: Similar file access times (PS sharing)
   - **Storage3**: Class2 optimized (cache hits)

5. **Scheduling Strategies**:
   - **FCFS**: WebServer, Storage1, Storage3 (first-come-first-served)
   - **PS**: Storage2 (processor sharing for fair file access)

6. **Performance Considerations**:
   - Feedback loops create multiplicative effects
   - WebServer becomes central bottleneck
   - Different classes compete for same resources
   - Complex interactions between service tiers

7. **Solver Selection**:
   - **CTMC**: May struggle with large state space
   - **MVA**: Good for networks with feedback
   - **MAM**: Matrix methods for open networks
   - **JMT**: Simulation handles complexity well

The model represents:
- Modern web applications with microservices
- Enterprise systems with multiple data tiers
- Cloud computing resource allocation
- Multi-tier application performance analysis

Key insight: Complex routing with feedback loops significantly amplifies the load on central components (WebServer), making capacity planning critical for system performance.